In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
# import osmnx as ox
import contextily as cx
import data_preprocessing as dp

In [2]:
years = range(2020, 2024)
data_list = []

# Schleife über jedes Jahr
for year in years:
    df = pd.read_csv(f"MVG_Rad_Fahrten_{year}.csv", sep=";", decimal=",", parse_dates=["STARTTIME       ", "ENDTIME         "])
    
    # Entfernen von Leerzeichen in den Columns
    column_names = [dp.remove_space(column) for column in df.columns]
    df.columns = column_names

    # Hinzufügen des eingelesenen DataFrames zur Liste
    data_list.append(df)

# alle Daten zu einem einzigen DataFrame kombinieren 
df = pd.concat(data_list, ignore_index=True)

/var/folders/pd/zg0xw_j14k3gvz5jqksfy67m0000gn/T/ipykernel_15955/1466124582.py:6: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"MVG_Rad_Fahrten_{year}.csv", sep=";", decimal=",", parse_dates=["STARTTIME       ", "ENDTIME         "])
/var/folders/pd/zg0xw_j14k3gvz5jqksfy67m0000gn/T/ipykernel_15955/1466124582.py:6: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"MVG_Rad_Fahrten_{year}.csv", sep=";", decimal=",", parse_dates=["STARTTIME       ", "ENDTIME         "])
/var/folders/pd/zg0xw_j14k3gvz5jqksfy67m0000gn/T/ipykernel_15955/1466124582.py:6: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f"MVG_Rad_Fahrten_{year}.csv", sep=";", decimal=",", parse_dates=["STARTTIME       ", "ENDTIME         "])


In [3]:
df.head()

,Row,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME
0,1,2020-01-01 00:10:00,2020-01-01 00:21:00,48.16957,11.58736,48.16568,11.56474,0,,0,
1,2,2020-01-01 00:14:00,2020-01-01 00:32:00,48.13652,11.5929,48.13502,11.57901,0,,0,
2,3,2020-01-01 00:25:00,2020-01-01 00:41:00,48.1706,11.54881,48.16562,11.5648,0,,0,
3,4,2020-01-01 00:29:00,2020-01-01 00:43:00,48.17433,11.56033,48.17326,11.59075,0,,0,
4,5,2020-01-01 00:29:00,2020-01-01 00:30:00,48.12354,11.53372,48.12374,11.53319,0,,0,


In [4]:
df.columns

Index(['Row', 'STARTTIME', 'ENDTIME', 'STARTLAT', 'STARTLON', 'ENDLAT',
       'ENDLON', 'RENTAL_IS_STATION', 'RENTAL_STATION_NAME',
       'RETURN_IS_STATION', 'RETURN_STATION_NAME'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760575 entries, 0 to 2760574
Data columns (total 11 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Row                  int64         
 1   STARTTIME            datetime64[ns]
 2   ENDTIME              datetime64[ns]
 3   STARTLAT             object        
 4   STARTLON             object        
 5   ENDLAT               object        
 6   ENDLON               object        
 7   RENTAL_IS_STATION    object        
 8   RENTAL_STATION_NAME  object        
 9   RETURN_IS_STATION    object        
 10  RETURN_STATION_NAME  object        
dtypes: datetime64[ns](2), int64(1), object(8)
memory usage: 231.7+ MB


In [6]:
# Entfernen von Leerzeichen bei Stationsnamen
df["RENTAL_STATION_NAME"] = df["RENTAL_STATION_NAME"].apply(dp.remove_space)
df["RETURN_STATION_NAME"] = df["RETURN_STATION_NAME"].apply(dp.remove_space)

In [ ]:
# STARTTIME und ENDTIME als datetime
df = dp.to_datetime(df)

,Row,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME
0,1,2020-01-01 00:10:00,2020-01-01 00:21:00,48.16957,11.58736,48.16568,11.56474,0,,0,
1,2,2020-01-01 00:14:00,2020-01-01 00:32:00,48.13652,11.5929,48.13502,11.57901,0,,0,
2,3,2020-01-01 00:25:00,2020-01-01 00:41:00,48.1706,11.54881,48.16562,11.5648,0,,0,
3,4,2020-01-01 00:29:00,2020-01-01 00:43:00,48.17433,11.56033,48.17326,11.59075,0,,0,
4,5,2020-01-01 00:29:00,2020-01-01 00:30:00,48.12354,11.53372,48.12374,11.53319,0,,0,
...,...,...,...,...,...,...,...,...,...,...,...
2760570,710102,2023-12-31 23:52:00,2023-12-31 23:57:00,48.16719,11.55854,48.16917,11.55547,0,,0,
2760571,710103,2023-12-31 23:52:00,2024-01-01 00:07:00,48.17061,11.57391,48.17334,11.55952,0,,0,
2760572,710104,2023-12-31 23:53:00,2023-12-31 23:57:00,48.14131,11.56144,48.14094,11.56044,0,,0,
2760573,710105,2023-12-31 23:54:00,2023-12-31 23:59:00,48.12353,11.54494,48.12674,11.54758,0,,0,


In [ ]:
# Löschen von "Row"
df = df.drop("Row", axis=1)

,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME
0,2020-01-01 00:10:00,2020-01-01 00:21:00,48.16957,11.58736,48.16568,11.56474,0,,0,
1,2020-01-01 00:14:00,2020-01-01 00:32:00,48.13652,11.59290,48.13502,11.57901,0,,0,
2,2020-01-01 00:25:00,2020-01-01 00:41:00,48.17060,11.54881,48.16562,11.56480,0,,0,
3,2020-01-01 00:29:00,2020-01-01 00:43:00,48.17433,11.56033,48.17326,11.59075,0,,0,
4,2020-01-01 00:29:00,2020-01-01 00:30:00,48.12354,11.53372,48.12374,11.53319,0,,0,
...,...,...,...,...,...,...,...,...,...,...
2760570,2023-12-31 23:52:00,2023-12-31 23:57:00,48.16719,11.55854,48.16917,11.55547,0,,0,
2760571,2023-12-31 23:52:00,2024-01-01 00:07:00,48.17061,11.57391,48.17334,11.55952,0,,0,
2760572,2023-12-31 23:53:00,2023-12-31 23:57:00,48.14131,11.56144,48.14094,11.56044,0,,0,
2760573,2023-12-31 23:54:00,2023-12-31 23:59:00,48.12353,11.54494,48.12674,11.54758,0,,0,


In [ ]:
# Formatierung der Koordinaten
# Entfernung ungültiger Daten
df["STARTLAT"] = df["STARTLAT"].apply(dp.format_coordinate)
df["STARTLAT"] = df["STARTLAT"].apply(dp.remove_invalid_latitudes)
df["STARTLON"] = df["STARTLON"].apply(dp.format_coordinate)
df["STARTLON"] = df["STARTLON"].apply(dp.remove_invalid_longitudes)
df["ENDLAT"] = df["ENDLAT"].apply(dp.format_coordinate)
df["ENDLAT"] = df["ENDLAT"].apply(dp.remove_invalid_latitudes)
df["ENDLON"] = df["ENDLON"].apply(dp.format_coordinate)
df["ENDLON"] = df["ENDLON"].apply(dp.remove_invalid_longitudes)

In [9]:
# Formatierung von is_station
df["RENTAL_IS_STATION"] = df["RENTAL_IS_STATION"].apply(dp.format_is_station)
df['RENTAL_IS_STATION'] = df['RENTAL_IS_STATION'].astype('Int64')
df["RETURN_IS_STATION"] = df["RETURN_IS_STATION"].apply(dp.format_is_station)
df['RETURN_IS_STATION'] = df['RETURN_IS_STATION'].astype('Int64')

Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             
Formatierung nicht möglich, setze Wert auf NA: NULL             


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2760575 entries, 0 to 2760574
Data columns (total 11 columns):
 #   Column               Dtype         
---  ------               -----         
 0   Row                  int64         
 1   STARTTIME            datetime64[ns]
 2   ENDTIME              datetime64[ns]
 3   STARTLAT             float64       
 4   STARTLON             float64       
 5   ENDLAT               float64       
 6   ENDLON               float64       
 7   RENTAL_IS_STATION    Int64         
 8   RENTAL_STATION_NAME  object        
 9   RETURN_IS_STATION    Int64         
 10  RETURN_STATION_NAME  object        
dtypes: Int64(2), datetime64[ns](2), float64(4), int64(1), object(2)
memory usage: 236.9+ MB


In [11]:
for column in df.columns:
    print(f"Anzahl NA-Werte in {column}: {df[column].isna().sum()}")

Anzahl NA-Werte in Row: 0
Anzahl NA-Werte in STARTTIME: 0
Anzahl NA-Werte in ENDTIME: 0
Anzahl NA-Werte in STARTLAT: 19671
Anzahl NA-Werte in STARTLON: 19649
Anzahl NA-Werte in ENDLAT: 24819
Anzahl NA-Werte in ENDLON: 24799
Anzahl NA-Werte in RENTAL_IS_STATION: 1969
Anzahl NA-Werte in RENTAL_STATION_NAME: 0
Anzahl NA-Werte in RETURN_IS_STATION: 2499
Anzahl NA-Werte in RETURN_STATION_NAME: 0


In [14]:
# Hinzufügen einer Spalte für die Dauer
df["DURATION"] = df["ENDTIME"] - df["STARTTIME"]

In [15]:
df

,STARTTIME,ENDTIME,STARTLAT,STARTLON,ENDLAT,ENDLON,RENTAL_IS_STATION,RENTAL_STATION_NAME,RETURN_IS_STATION,RETURN_STATION_NAME,DURATION
0,2020-01-01 00:10:00,2020-01-01 00:21:00,48.16957,11.58736,48.16568,11.56474,0,,0,,0 days 00:11:00
1,2020-01-01 00:14:00,2020-01-01 00:32:00,48.13652,11.59290,48.13502,11.57901,0,,0,,0 days 00:18:00
2,2020-01-01 00:25:00,2020-01-01 00:41:00,48.17060,11.54881,48.16562,11.56480,0,,0,,0 days 00:16:00
3,2020-01-01 00:29:00,2020-01-01 00:43:00,48.17433,11.56033,48.17326,11.59075,0,,0,,0 days 00:14:00
4,2020-01-01 00:29:00,2020-01-01 00:30:00,48.12354,11.53372,48.12374,11.53319,0,,0,,0 days 00:01:00
...,...,...,...,...,...,...,...,...,...,...,...
2760570,2023-12-31 23:52:00,2023-12-31 23:57:00,48.16719,11.55854,48.16917,11.55547,0,,0,,0 days 00:05:00
2760571,2023-12-31 23:52:00,2024-01-01 00:07:00,48.17061,11.57391,48.17334,11.55952,0,,0,,0 days 00:15:00
2760572,2023-12-31 23:53:00,2023-12-31 23:57:00,48.14131,11.56144,48.14094,11.56044,0,,0,,0 days 00:04:00
2760573,2023-12-31 23:54:00,2023-12-31 23:59:00,48.12353,11.54494,48.12674,11.54758,0,,0,,0 days 00:05:00
